# CS280 Programming Assignment 3
__Comparison of Boosted Perceptrons and SVM__<br>
<br>
Compiler: Python 3.6.5<br>
OS: Windows 7

## 2. Perceptron Classifier Construction

### 2.1. Generate data
Generate data consisting of 100 two-dimensional vectors taken from a normal distribution with $\mu_{1} = [0,0]^{T}$, $\sigma_{1} = I$ and label them as class "$-1$"<br>
<br>
Form a class "$-1$" training subset by setting aside 50 points and let the remaining 50 points serve as part of the test set.

In [309]:
import numpy as np

def generate_random_data(mean, standard_dev, data_size, train_test_split, label):
    data = np.random.normal(mean_1, standard_dev_1, size=data_size)
    
    train_data = data[ :int(train_test_split*data.shape[0])]
    train_labels = label * np.ones(train_data.shape[0])
    
    test_data = data[int(train_test_split*data.shape[0]): ]
    test_labels = label * np.ones(test_data.shape[0])
    
    return train_data, train_labels, test_data, test_labels
    

In [310]:
NUM_DATA_POINTS = 100
NUM_DATA_DIM = 2

mean = 0
std = 1

classneg1_train_data, classneg1_train_labels, classneg1_test_data, classneg1_test_labels = generate_random_data(mean, std, [NUM_DATA_POINTS, NUM_DATA_DIM], 0.5, -1)

Do the same for class "+1" for another normal distribution with $\mu_{2} = [0,0]^{T}$, $\sigma_{2} = I$. <br>

In [311]:
classpos1_train_data, classpos1_train_labels, classpos1_test_data, classpos1_test_labels = generate_random_data(mean, std, [NUM_DATA_POINTS, NUM_DATA_DIM], 0.5, 1)

Merge the subsets to form the training and test sets.

In [312]:
def shuffle_data(data, labels):
    indices = np.arange(data.shape[0])
    np.random.shuffle(indices)
    data = data[indices, :]
    labels = labels[indices]
    return data, labels


In [313]:
train_set, train_labels = shuffle_data(np.concatenate((classneg1_train_data, classpos1_train_data)), np.concatenate((classneg1_train_labels, classpos1_train_labels)))
test_set, test_labels = shuffle_data(np.concatenate((classneg1_test_data, classpos1_test_data)), np.concatenate((classneg1_test_labels, classpos1_test_labels)))

### 2.2. Write code _classify(...)_
Write code named _classify(...)_ that implements the Pocket Algorithm for perceptron learning.<br>
Set _maxitercnt_ to 10000 iterations.

In [327]:
import random

def classify(train_set, train_labels, maxitercnt):
    # Number of features, d
    d = train_set.shape[1]

    # Number of training samples
    N = train_set.shape[0]

    # Number of consecutive iterations for which v correctly classified the examples
    n_v = 0

    # Number of consecutive iterations for which w correctly classified the examples
    n_w = 0

    w = np.random.uniform(low=-0.1, high=0.1, size=(d+1, 1))
    v = w

    for itercnt in range(maxitercnt):
        print('@Iteration # %d of %d'%(itercnt+1, maxitercnt))
        random_index = random.sample(range(N), 1)
        x_j = np.insert(train_set[random_index], 0, 1)
        y_j = train_labels[random_index]

        y_j_hat = 1 if (np.inner(v.transpose(), x_j) >= 0) else -1

        if y_j*y_j_hat > 0:
            n_v += 1
            print('\tweight vector v correctly predicted label!')
        else:
            print('\tweight vector v incorrectly predicted label! Putting in pocket...')
            if n_v > n_w:
                print('\t\tFOUND NEW BEST W!')
                w = v
                n_w = n_v
            v = v + (y_j*x_j).reshape(-1, 1)
            n_v = 0
        print('\tn_v=%d, n_w=%d'%(n_v, n_w))
        
    return w

In [328]:
maxitercnt = 10000
pocket_w = classify(train_set, train_labels, maxitercnt)

@Iteration # 1 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=0
@Iteration # 2 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=0
@Iteration # 3 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=0
@Iteration # 4 of 10000
	weight vector v correctly predicted label!
	n_v=2, n_w=0
@Iteration # 5 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
		FOUND NEW BEST W!
	n_v=0, n_w=2
@Iteration # 6 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=2
@Iteration # 7 of 10000
	weight vector v correctly predicted label!
	n_v=2, n_w=2
@Iteration # 8 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=2
@Iteration # 9 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=2
@Iteration # 10 of 10000
	weight vector v correctly predicted label!
	n_v=2, n_w=2
@Iteration # 11 of 10000
	weight vector v incorrectly predicted lab

	weight vector v correctly predicted label!
	n_v=1, n_w=10
@Iteration # 540 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 541 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 542 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 543 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 544 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 545 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=10
@Iteration # 546 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 547 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=10
@Iteration # 548 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 54

	n_v=5, n_w=10
@Iteration # 873 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 874 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=10
@Iteration # 875 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 876 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=10
@Iteration # 877 of 10000
	weight vector v correctly predicted label!
	n_v=2, n_w=10
@Iteration # 878 of 10000
	weight vector v correctly predicted label!
	n_v=3, n_w=10
@Iteration # 879 of 10000
	weight vector v correctly predicted label!
	n_v=4, n_w=10
@Iteration # 880 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 881 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 882 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=10
@Iteration # 883 of 10000
	weight vector v 

	n_v=0, n_w=10
@Iteration # 1206 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 1207 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 1208 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=10
@Iteration # 1209 of 10000
	weight vector v correctly predicted label!
	n_v=2, n_w=10
@Iteration # 1210 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 1211 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 1212 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=10
@Iteration # 1213 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 1214 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=10
@Iteration # 1215 of 10000
	weight vector v correctly predicted label!
	n

	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=13
@Iteration # 1706 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=13
@Iteration # 1707 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=13
@Iteration # 1708 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=13
@Iteration # 1709 of 10000
	weight vector v correctly predicted label!
	n_v=2, n_w=13
@Iteration # 1710 of 10000
	weight vector v correctly predicted label!
	n_v=3, n_w=13
@Iteration # 1711 of 10000
	weight vector v correctly predicted label!
	n_v=4, n_w=13
@Iteration # 1712 of 10000
	weight vector v correctly predicted label!
	n_v=5, n_w=13
@Iteration # 1713 of 10000
	weight vector v correctly predicted label!
	n_v=6, n_w=13
@Iteration # 1714 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=13
@Iteration # 1715 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=13
@Iteration #

...

@Iteration # 7703 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=13
@Iteration # 7704 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=13
@Iteration # 7705 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=13
@Iteration # 7706 of 10000
	weight vector v correctly predicted label!
	n_v=2, n_w=13
@Iteration # 7707 of 10000
	weight vector v correctly predicted label!
	n_v=3, n_w=13
@Iteration # 7708 of 10000
	weight vector v correctly predicted label!
	n_v=4, n_w=13
@Iteration # 7709 of 10000
	weight vector v correctly predicted label!
	n_v=5, n_w=13
@Iteration # 7710 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=13
@Iteration # 7711 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=13
@Iteration # 7712 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=13
@Iteration # 7713 of 1000

	weight vector v correctly predicted label!
	n_v=1, n_w=24
@Iteration # 8203 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 8204 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 8205 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=24
@Iteration # 8206 of 10000
	weight vector v correctly predicted label!
	n_v=2, n_w=24
@Iteration # 8207 of 10000
	weight vector v correctly predicted label!
	n_v=3, n_w=24
@Iteration # 8208 of 10000
	weight vector v correctly predicted label!
	n_v=4, n_w=24
@Iteration # 8209 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 8210 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=24
@Iteration # 8211 of 10000
	weight vector v correctly predicted label!
	n_v=2, n_w=24
@Iteration # 8212 of 10000
	weight vector v correctly predicted label!
	n_v=3, n_w=24
@Iteration #

	weight vector v correctly predicted label!
	n_v=1, n_w=24
@Iteration # 8703 of 10000
	weight vector v correctly predicted label!
	n_v=2, n_w=24
@Iteration # 8704 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 8705 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 8706 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=24
@Iteration # 8707 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 8708 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=24
@Iteration # 8709 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 8710 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 8711 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 8712 of 10000
	w

@Iteration # 9202 of 10000
	weight vector v correctly predicted label!
	n_v=2, n_w=24
@Iteration # 9203 of 10000
	weight vector v correctly predicted label!
	n_v=3, n_w=24
@Iteration # 9204 of 10000
	weight vector v correctly predicted label!
	n_v=4, n_w=24
@Iteration # 9205 of 10000
	weight vector v correctly predicted label!
	n_v=5, n_w=24
@Iteration # 9206 of 10000
	weight vector v correctly predicted label!
	n_v=6, n_w=24
@Iteration # 9207 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 9208 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=24
@Iteration # 9209 of 10000
	weight vector v correctly predicted label!
	n_v=2, n_w=24
@Iteration # 9210 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 9211 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=24
@Iteration # 9212 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...


@Iteration # 9702 of 10000
	weight vector v correctly predicted label!
	n_v=4, n_w=24
@Iteration # 9703 of 10000
	weight vector v correctly predicted label!
	n_v=5, n_w=24
@Iteration # 9704 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 9705 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 9706 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=24
@Iteration # 9707 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 9708 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=24
@Iteration # 9709 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 9710 of 10000
	weight vector v correctly predicted label!
	n_v=1, n_w=24
@Iteration # 9711 of 10000
	weight vector v incorrectly predicted label! Putting in pocket...
	n_v=0, n_w=24
@Iteration # 9712 of 1000

In [329]:
w

array([[ 0.04570159],
       [ 0.05757838],
       [-0.04087867]])

### 2.3 Write code _predict()_
Write code named _predict(...)_ to test the classifier and  measure the sum of square errors for the test set.

In [323]:
def predict(input_data, w, true_labels):
    x_j = np.insert(input_data, 0, 1, axis=1)
    
    predicted_labels = np.zeros((input_data.shape[0], 1))
    for index in range(x_j.shape[0]):
        prod = np.inner(w.transpose(), x_j[index, :].reshape(1,-1))
        predicted_labels[index] = 1 if (prod >= 0) else -1
        
    error = (1/len(true_labels))*np.sum(np.square(true_labels - predicted_labels))
    return predicted_labels, error

In [325]:
predicted_labels, error = predict(test_set, pocket_w, test_labels)

## 3. Adaboost Construction and Evaluation


### 3.1. Write code _adabtrain()_
Write code called _adabtrain()_ that implements the Adaboost algorithm with the Pocket Algorithm as the basic learner.

### 3.2. Form the training set
Form the training set bys ampling 400 points from the banana dataset provided. Use the remaining 4900 points as your test set.

### 3.3. Write code named _adabpredict()_
Write code named adabpredict() that classifies unknown/unseen data

### 3.4. Train the boosted perceptron algorithm
Using _adabtrain()_ and _adabpredict()_, train the boosted perceptron algorithm and measure the training and test accuracies for the ensemble classifier having _K_ learners.

### 3.5. Write code that automatically plots accuracies
Write code that automatially plots the training and test accuracies against the number of learners _K_ used for training. Use _K_ = 10, 20, 30, ...., 1000

### 3.6. Train and test your Boosted Perceptron on the splice dataset
Train and test you Boosted perceptron on the splice dataset and plot the accuracy vs _K_ curves for both training and test. Use 1000 points for training, 2175 points for test, and K = 10, 20, 30, ..., 1000

## SVM Classifier